# 3F7 Lab: CamZIP

## Tree data structures

Import all functions from the package trees where we put together a number of tools for handling trees in the 3F7 lab.

In [1]:
from trees import *
from pprint import pprint

Now define a simple tree (play around with this command and construct more complicated trees....)

In [2]:
t = [-1, 0, 0, 1, 1, 3, 3, 4, 2] 

The following command will print a string that can be copy-pasted into a tree visualising website like [phylo.io](https://phylo.io) (don't forget to add a new line at the end of the string after cutting and pasting)

In [3]:
print(tree2newick(t))
print('Cut and paste the string on the previous line and add a "new line" at the end of the string.')

((( ,),()),())
Cut and paste the string on the previous line and add a "new line" at the end of the string.


You can also add labels to the nodes in the `tree2newick` command.

In [4]:
print(tree2newick(t,['root', 'child 0', 'grandchild 0', 'grandchild 1', 'child 1']))

(((root,child 0)7,(grandchild 0)8)5,(grandchild 1)6)child 1


If there are less labels than nodes, the labels will be interpreted "leaves first"

In [5]:
print(tree2newick(t,['symbol 0','symbol 1', 'symbol 2']))

(((symbol 0,symbol 1)7,(symbol 2)8)5,(3)6)4


The following command converts a variable-length code described by a tree to a code table format.

In [6]:
print(tree2code(t))

{'0': [0, 0, 0], '1': [0, 0, 1], '2': [0, 1, 0], '3': [1, 0]}


Verify that the inverse function can recover the tree. 

In [7]:
print(code2tree(tree2code(t)))

[-1, 0, 1, 2, 2, 1, 5, 0, 7]


But the following may happen as well. Can you explain why?

In [8]:
print(code2tree(tree2code([3,3,4,4,-1])))

[-1, 0, 1, 1, 0]


In [9]:
print(tree2newick([3,3,4,4,-1], ['grandchild 0', 'grandchild 1', 'child 0', 'child 1', 'root']))

(child 0,(grandchild 0,grandchild 1)child 1)root


Similarly but far more problematic is the following inversion. The resulting assignment of codeword to symbols is fundamentally different from the original and would result in wrong decoding.

In [10]:
print(tree2code(code2tree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'0': [0], '1': [1, 0], '2': [1, 1, 0], '3': [1, 1, 1]}


These problems are all solved when using the extended tree format.

In [11]:
xt = tree2xtree([3,3,4,4,-1], ['a', 'b', 'c'])
print(xt)

[[3, [], 'a'], [3, [], 'b'], [4, [], 'c'], [4, [0, 1], '3'], [-1, [2, 3], '4']]


In [12]:
print(xtree2code(code2xtree({'0':[1], '1':[0,1], '2':[0,0,1], '3':[0,0,0]})))

{'0': [1], '1': [0, 1], '2': [0, 0, 1], '3': [0, 0, 0]}


## Testing your Shannon-Fano Code

This next section can only be completed once you have a working Shannon-Fano function `shannon_fano()`

In [13]:
from vl_codes import shannon_fano
from random import random
p = [random() for k in range(16)]
p = dict([(chr(k+ord('a')),p[k]/sum(p)) for k in range(len(p))])
print(f'Probability distribution: {p}\n')
c = shannon_fano(p)
print(f'Codebook: {c}\n')
xt = code2xtree(c)
print(f'Cut and paste for phylo.io: {xtree2newick(xt)}')

Probability distribution: {'a': 0.00463771964759533, 'b': 0.021117261878694168, 'c': 0.11007506088918036, 'd': 0.059812148433953646, 'e': 0.11996123283566233, 'f': 0.014303536848789527, 'g': 0.0872458199234202, 'h': 0.11369981576208267, 'i': 0.09065832191684839, 'j': 0.09805247655899081, 'k': 0.07130906895129181, 'l': 0.07081520980491592, 'm': 0.07791021072123963, 'n': 0.04326549738705187, 'o': 0.00014756968372099972, 'p': 0.01698904875656237}

Codebook: {'e': [0, 0, 0, 0], 'h': [0, 0, 0, 1], 'c': [0, 0, 1, 1], 'j': [0, 1, 0, 1], 'i': [0, 1, 1, 1], 'g': [1, 0, 0, 0], 'm': [1, 0, 0, 1], 'k': [1, 0, 1, 1], 'l': [1, 1, 0, 0], 'd': [1, 1, 0, 1, 0], 'n': [1, 1, 1, 0, 0], 'b': [1, 1, 1, 1, 0, 0], 'p': [1, 1, 1, 1, 0, 1], 'f': [1, 1, 1, 1, 1, 0, 1], 'a': [1, 1, 1, 1, 1, 1, 1, 0], 'o': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]}

Cut and paste for phylo.io: ((((e,h)3,(c)4)2,((j)6,(i)7)5)1,(((g,m)10,(k)11)9,((l,(d)14)13,((n)16,((b,p)18,((f)20,((a,(((((o)27)26)25)24)23)22)21)19)17)15)12)8)0


We can upload data from a file, for example `hamlet.txt`, and display the first few lines...

In [5]:
f = open('hamlet.txt', 'r')
hamlet = f.read()
f.close()
print(hamlet[:294])

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now compute the startistics of the file:

In [15]:
from itertools import groupby
frequencies = dict([(key, len(list(group))) for key, group in groupby(sorted(hamlet))])
Nin = sum([frequencies[a] for a in frequencies])
p = dict([(a,frequencies[a]/Nin) for a in frequencies])
print(f'File length: {Nin}')

File length: 207039


We can view the alphabet of symbols used in the file:

In [16]:
print(list(p))

['\n', ' ', '!', '&', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']


We are now ready to construct the Shannon-Fano code for this file, and view its tree (cut and paste into [phylo.io](https://phylo.io), don't forget to add a carriage return at the end, click on "Branch Labels/Support" under "Settings", then right-click on the root of the tree and select "expand all". 

In [17]:
c = shannon_fano(p)
print(xtree2newick(code2xtree(c)))

((space,((e,(t)4)3,((o)6,(a,s)7)5)2)1,((((n)11,(h,i)12)10,((r,(carriage return)15)14,((l)17,(d)18)16)13)9,((((u,m)22,(comma,(y)24)23)21,(((w)27,(f)28)26,((c,g)30,(p)31)29)25)20,(((((A)36,(b,T)37)35,((I)39,(E)40)38)34,(((.)43,(L)44)42,((v)46,(k,O)47)45)41)33,((((')51,(H,R)52)50,((N,(S)55)54,((U)57,(M)58)56)53)49,((((semi-colon,colon)62,(D)63)61,((C,G)65,(W,?)66)64)60,(((-,(P)70)69,((!)72,(left square bracket,right square bracket)73)71)68,(((B,F)76,((x,K)78)77)75,(((q)81,(Y,(j)83)82)80,(((Q)86,(Z)87)85,((z,(vertical bar)90)89,((V)92,((left parenthesis,right parenthesis)94,((J)96,((&)98)97)95)93)91)88)84)79)74)67)59)48)32)19)8)0


In [18]:
print(c)

{' ': [0, 0], 'e': [0, 1, 0, 0], 't': [0, 1, 0, 1, 0], 'o': [0, 1, 1, 0, 0], 'a': [0, 1, 1, 1, 0], 's': [0, 1, 1, 1, 1], 'n': [1, 0, 0, 0, 0], 'h': [1, 0, 0, 1, 0], 'i': [1, 0, 0, 1, 1], 'r': [1, 0, 1, 0, 0], '\n': [1, 0, 1, 0, 1, 1], 'l': [1, 0, 1, 1, 0, 0], 'd': [1, 0, 1, 1, 1, 0], 'u': [1, 1, 0, 0, 0, 0], 'm': [1, 1, 0, 0, 0, 1], ',': [1, 1, 0, 0, 1, 0], 'y': [1, 1, 0, 0, 1, 1, 1], 'w': [1, 1, 0, 1, 0, 0, 1], 'f': [1, 1, 0, 1, 0, 1, 0], 'c': [1, 1, 0, 1, 1, 0, 0], 'g': [1, 1, 0, 1, 1, 0, 1], 'p': [1, 1, 0, 1, 1, 1, 0], 'A': [1, 1, 1, 0, 0, 0, 0, 0], 'b': [1, 1, 1, 0, 0, 0, 1, 0], 'T': [1, 1, 1, 0, 0, 0, 1, 1], 'I': [1, 1, 1, 0, 0, 1, 0, 1], 'E': [1, 1, 1, 0, 0, 1, 1, 1], '.': [1, 1, 1, 0, 1, 0, 0, 1], 'L': [1, 1, 1, 0, 1, 0, 1, 1], 'v': [1, 1, 1, 0, 1, 1, 0, 0], 'k': [1, 1, 1, 0, 1, 1, 1, 0], 'O': [1, 1, 1, 0, 1, 1, 1, 1], "'": [1, 1, 1, 1, 0, 0, 0, 0], 'H': [1, 1, 1, 1, 0, 0, 1, 0], 'R': [1, 1, 1, 1, 0, 0, 1, 1], 'N': [1, 1, 1, 1, 0, 1, 0, 0], 'S': [1, 1, 1, 1, 0, 1, 0, 1, 0], 'U':

Now we can actually encode the file `hamlet.txt` using the Shannon-Fano code we constructed.

In [19]:
from vl_codes import vl_encode
hamlet_sf = vl_encode(hamlet, c)
print(f'Length of binary sequence: {len(hamlet_sf)}')

Length of binary sequence: 997548


We have commands to convert a bit sequence into a byte sequence (including a 3 bit prefix that helps us determine the length of the bit sequence):

In [20]:
from vl_codes import bytes2bits, bits2bytes
x = bits2bytes([0,1])
print([format(a, '08b') for a in x])
y = bytes2bits(x)
print(f'The original bits are: {y}')

['01101000']
The original bits are: [0, 1]


In [21]:
test = bits2bytes([0,1,1,0,1,1,0,0,0])
print(test)

[141, 128]


We now apply the bits to byte conversion to the compressed text of Hamlet to compute the length of the compressed file.

In [22]:
hamlet_zipped = bits2bytes(hamlet_sf)
Nout = len(hamlet_zipped)
print(f'Length of compressed string: {Nout}')

Length of compressed string: 124694


The compression ratio can be expressed in two ways, unitless or in bits/bytes:

In [23]:
print(f'Compression ratio (rateless): {Nout/Nin}')
print(f'Compression ratio (bits per byte): {8.0*Nout/Nin}')

Compression ratio (rateless): 0.6022730017049928
Compression ratio (bits per byte): 4.818184013639942


The lower bound for compression is the Entropy, measured in bits, that can be computed using an in-line function in Python:

In [24]:
from math import log2
H = lambda pr: -sum([pr[a]*log2(pr[a]) for a in pr])
print(f'Entropy: {H(p)}')

Entropy: 4.449863631694343


We now proceed to decode the compressed Hamlet sequence

In [25]:
from vl_codes import vl_decode
xt = code2xtree(c)
hamlet_unzipped = vl_decode(hamlet_sf, xt)
print(f'Length of unzipped file: {len(hamlet_unzipped)}')

Length of unzipped file: 207039


We can view the first few lines of the input (note the command `join` that turns the list of strings into one string)

In [26]:
print(''.join(hamlet_unzipped[:294]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


## Compressing and uncompressing files

This is where we put it all together, compressing directly from input to output file. Play around with these commands once you implemented Huffman coding and arithmetic coding. We begin by importing the compression and decompression functions.

In [27]:
from camzip import camzip
from camunzip import camunzip

The next commands define the method to be used and the filename. Modify those when you are trying other methods on various files. 

In [28]:
method = 'huffman'
filename = 'hamlet.txt'
probability_on_the_go = True

Now we do the actual compression and decompression...

In [29]:
camzip(method, filename, probability_on_the_go)
camunzip(filename + '.cz' + method[0], probability_on_the_go)

The next few lines perform various statistical measurements and verifies that the decompressed file is identical to the compressed file.

In [8]:
from filecmp import cmp
from os import stat
from json import load
Nin = stat(filename).st_size
print(f'Length of original file: {Nin} bytes')
Nout = stat(filename + '.cz' + method[0]).st_size
print(f'Length of compressed file: {Nout} bytes')
print(f'Compression rate: {8.0*Nout/Nin} bits/byte')
with open(filename + '.czp', 'r') as fp:
    freq = load(fp)
pf = dict([(a, freq[a]/Nin) for a in freq])
print(f'Entropy: {H(pf)} bits per symbol')
if cmp(filename,filename+'.cuz'):
    print('The two files are the same')
else:
    print('The files are different')

NameError: name 'filename' is not defined

In [31]:
import json
json.dump(p, open("probabilities.json", "w"))
probability_size = stat("probabilities.json").st_size
print("Size of the probabilities is {} bytes.".format(probability_size))

Size of the probabilities is 1825 bytes.


## Huffman coding

This section will only work once you have a working function `huffman()`. We first repeat the tree construction and visualisation.

In [32]:
from vl_codes import huffman
xt = huffman(p)
print(xtree2newick(xt))

((((a,(((v,L)96,c)105,d)113)121,(((f,((M,(P,((Z,(V,(right parenthesis,((&,J)67,left parenthesis)68)69)70)72,x)76)81)88,.)97)106,l)114,o)122)127,((t,((w,((U,S)89,E)98)107,carriage return)115)123,((((I,T)99,y)108,((b,A)100,comma)109)116,(((((((Q,(vertical bar,z)71)73,(j,Y)74)77,-)82,N)90,p)101,(((?,W)83,R)91,((G,(F,B)78)84,H)92)102)110,r)117)124)128)130,(space,((e,(i,(m,((',(C,(left square bracket,right square bracket)79)85)93,g)103)111)118)125,((h,n)119,(s,(u,(((D,colon)86,O)94,(k,(((q,K)75,!)80,semi-colon)87)95)104)112)120)126)129)131)132


Observe how the Huffman tree differs from the Shannon-Fano tree. What are its shortest and its longest codeword? You can use the `camzip` code above changing the method to `'huffman'` to test the compression rate etc. You may also want to do it by hand to test the error resilience:

In [33]:
c = xtree2code(xt)
hamlet_huf = vl_encode(hamlet, c)
hamlet_decoded = vl_decode(hamlet_huf, xt)
print(''.join(hamlet_decoded[:294]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. (KING CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


We now introduce a random bit flip (bit 400 flipped) in the compressed sequence and observe the result.

In [34]:
hamlet_corrupted = hamlet_huf.copy()
hamlet_corrupted[400] ^= 1
hamlet_decoded = vl_decode(hamlet_corrupted, xt)
print(''.join(hamlet_decoded[:297]))

        HAMLET


        DRAMATIS PERSONAE


CLAUDIUS        king of Denmark. y,; 
aNG CLAUDIUS:)

HAMLET  son to the late, and nephew to the present king.

POLONIUS        lord chamberlain. (LORD POLONIUS:)

HORATIO friend to Hamlet.

LAERTES son to Polonius.

LUCIANUS        nephew to the king.


## Arithmetic coding

We first try "by hand" to operate the steps of arithmetic coding using floating point numbers. We first compute the cumulative probability distribution.

In [35]:
f = [0.0]
for a in p:
    f.append(f[-1]+p[a])
f.pop()
f = dict([(a,f[k]) for a,k in zip(p,range(len(p)))])

We now perform by hand the first `n=4` steps of arithmetic coding. Vary `n` to observe the loss of precision. 

In [36]:
lo, hi = 0.0, 1.0
n = 4
for k in range(n):
    a = hamlet[k]
    lohi_range = hi - lo
    hi = lo + lohi_range * (f[a] + p[a])
    lo = lo + lohi_range * f[a]
print(f'lo = {lo}, hi = {hi}, hi-lo = {hi-lo}')

lo = 0.03994572862162076, hi = 0.04551184792510039, hi-lo = 0.005566119303479632


The output sequence is roughly the binary expression of `lo` (not exactly) and we can compute and observe it. What length `ell` would we need when encoding all of Hamlet?

In [37]:
from math import floor, ceil
ell = ceil(-log2(hi-lo))+2 if hi-lo > 0.0 else 96
print(bin(floor(lo*2**ell)))

0b101000


We encode and decode Hamlet again using arithmetic coding and verify that the first few lines of the play look as expected.

In [38]:
# import arithmetic as arith
# arith_encoded = arith.encode(hamlet, p)
# arith_decoded = arith.decode(arith_encoded, p, Nin)
# print('\n'+''.join(arith_decoded[:294]))

In [39]:
# import arithmetic as arith
# arith_encoded = arith.encode(hamlet, p, True)
# arith_decoded = arith.decode(arith_encoded, p, Nin, True)
# print('\n'+''.join(arith_decoded[:294]))

We now repeat the steps above but introduce a one bit mistake (bit 399 flipped) and observe the effect on the decoded text. Repeat this experiment varying the location of the mistake or adding more than one mistake. What do you observe? Can you explain why?

In [40]:
# arith_corrupted = arith_encoded.copy()
# arith_corrupted[399] ^= 1
# arith_corrupted[956] ^= 1
# arith_decoded = arith.decode(arith_corrupted, p, Nin)
# print('\n'+''.join(arith_decoded[:294]))

In [12]:
import time
import json
from contextual import full_encode
starting_time = time.time()
y, x, conditional_probs = full_encode(hamlet, n_gram=3)
print("Total time to encode and decode: {}s".format(time.time()-starting_time))

['\n', ' ', '!', '&', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|']


KeyboardInterrupt: 

In [10]:
from vl_codes import bytes2bits, bits2bytes
hamlet_compressed = bits2bytes(y)
print(len(hamlet_compressed))

89176


In [11]:
print(x)

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'H', 'A', 'M', 'L', 'E', 'T', '\n', '\n', '\n', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'D', 'R', 'A', 'M', 'A', 'T', 'I', 'S', ' ', 'P', 'E', 'R', 'S', 'O', 'N', 'A', 'E', '\n', '\n', '\n', 'C', 'L', 'A', 'U', 'D', 'I', 'U', 'S', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'k', 'i', 'n', 'g', ' ', 'o', 'f', ' ', 'D', 'e', 'n', 'm', 'a', 'r', 'k', '.', ' ', '(', 'K', 'I', 'N', 'G', ' ', 'C', 'L', 'A', 'U', 'D', 'I', 'U', 'S', ':', ')', '\n', '\n', 'H', 'A', 'M', 'L', 'E', 'T', ' ', ' ', 's', 'o', 'n', ' ', 't', 'o', ' ', 't', 'h', 'e', ' ', 'l', 'a', 't', 'e', ',', ' ', 'a', 'n', 'd', ' ', 'n', 'e', 'p', 'h', 'e', 'w', ' ', 't', 'o', ' ', 't', 'h', 'e', ' ', 'p', 'r', 'e', 's', 'e', 'n', 't', ' ', 'k', 'i', 'n', 'g', '.', '\n', '\n', 'P', 'O', 'L', 'O', 'N', 'I', 'U', 'S', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'l', 'o', 'r', 'd', ' ', 'c', 'h', 'a', 'm', 'b', 'e', 'r', 'l', 'a', 'i', 'n', '.', ' ', '(', 'L', 'O', 'R', 'D', ' ', 'P', 'O', 'L', 'O',